# Smurf Predictor | Brandon Jackson || Richard Tang
# This ambitious data science experiment's goal is to: given an input (summoner name), return a list of possible smurfs of that account. This project as inspried by an interview I watched with Team Liquid's coach Steve mentioned that when teams are scouting players they often need to get a full list of smurf accounts.
****
# For those of you who do not play the game League of Ledgends, often times a high elo (the rating system for the game) create smurf accounts to practice new champions or get lower queue time (since they must be matched up with players of their skill level as you climb the elo ladder you have fewer players as good as you). You may also want to hide a pocket pick you may bring out in the LCS/LCK/LPL/LMS/etc...(tournment) So there is insentive for coaches to want to know who other players may be practising on as well as recruits they may be scouting and want to get a fuller picture. Out of respect for player's who want to hide their smurf some code that would be avalible to pulbic on github will not be posted. However, if you are a coach or team looking for software like this please contact me at Brandon@devbjax.com(Prefered) || Bjax@cs.umd.edu.

imporvements, find know list of player smurf to get data like off role champoins and builds

In [0]:
!pip install riotwatcher

    100% |████████████████████████████████| 51kB 3.3MB/s 


# Test the API wrapper

In [0]:
from riotwatcher import RiotWatcher
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize
import time

def parse_get(url):
  response = requests.request("GET", url)
  return json.loads(response.text)

def get_player(summonerName):
  api_key_raw = 'RGAPI-a8a29452-1047-4cba-8046-395d72b01095'
  api_key = "?api_key=" + str(api_key_raw)
  #Need to generate a new riot API key every 24 hours
  watcher = RiotWatcher(api_key_raw)

  my_region = 'na1'
  #summonerName = 'Doublelift'
  me = watcher.summoner.by_name(my_region, summonerName)
  #print(me)

  account_id = str(me['accountId'])
  #Get list of match ID's by account
  url_matchlists = "https://na1.api.riotgames.com/lol/match/v3/matchlists/by-account/" + account_id + api_key
  match_list = parse_get(url_matchlists)

  matches = []
  lanes = []
  for i in range(match_list['endIndex']):
    matches.append(match_list['matches'][i]['gameId'])
    lanes.append(match_list['matches'][i]['lane'])
  ''' a full match look like this
  {'champion': 64,
   'gameId': 2915177882,
   'lane': 'JUNGLE',
   'platformId': 'NA1',
   'queue': 420,
   'role': 'NONE',
   'season': 11,
   'timestamp': 1542855765895}
  '''

  data_list = []
  counter = 0
  for i in matches:
    time.sleep(3)
    counter = counter + 1
    display(i, counter)
    #Get match details from matchID from matchlist
    url_match = "https://na1.api.riotgames.com/lol/match/v4/matches/"+ str(i) + api_key 
    url_timeline = "https://na1.api.riotgames.com/lol/match/v4/timelines/by-match/"+ str(i) + api_key

    #Cross reffrence so that we can figure out who is in timeslines since timeliensv4 does not give summer information
    #However we can use another repsonse to get it using matchesv4
    response = requests.request("GET", url_match)
    parsed_json = json.loads(response.text)
    index = 0
    for j in range(10):
       if summonerName == parsed_json['participantIdentities'][j]['player']['summonerName']:
          index = j
          break

    participantId = parsed_json['participantIdentities'][index]['participantId']
    parsed_json = parse_get(url_timeline)
    #display(parsed_json)
    #Filter dataframe for given summer
    data = json_normalize(parsed_json['frames'])
    filter_list = list(data)
    res = [k for k in filter_list if 'participantFrames.'+str(participantId)+'.' in k]
    res
    data = data[res]
    data_list.append(data)

  return data_list

In [0]:
list = get_player("Doublelift")#2863522938

2915177882

1

2915158637

2

2915117762

...

98

2847881802

99

2847266651

100

In [0]:
from google.colab import files
counter = 0
for i in list:
  counter += 1
  i.to_csv('game'+str(counter)+'.csv')
  files.download('game'+str(counter)+'.csv')

### If you only need to know, which champions a summoner plays in general a good approximation would be to use the champion-mastery scores

### /lol/champion-mastery/v3/scores/by-summoner/{summonerId}
****
### For exact numbers, you can get the matchlist entries, as they include the champion id played

### /lol/match/v3/matchlists/by-account/{accountId}

### /lol/spectator/v3/active-games/by-summoner/{summonerId} Get current game information for the given summoner ID

Use http://api.champion.gg/docs/ to compare common builds and find outlier items and build paths

****
### Get the match history then use that match id to get match spefics
Use /lol/match/v3/matchlists/by-account/{accountId}

passed the returned match history
https://na1.api.riotgames.com/lol/match/v3/timelines/by-match/2915177882


# EDA (Explority Data Analysis)

Possible Fun stuff
https://medium.com/pandascore-stories/league-of-legends-getting-champion-coordinates-from-the-minimap-using-deep-learning-48a49d35bb74